In [ ]:
import xarray as xr
import hvplot.xarray
import holoviews as hv
import numpy as np
from datetime import datetime

In [ ]:
# This function downloads and stores all netcdf files from the Aqua-Modis Satellite for Sable Island, Canada from April 12th to May 2nd, 2016 into a folder called 'data' 
%run -m phyto_voc.function

In [ ]:
# To find the proportion of cloud-covered pixels for each file in the 'data' folder we must create a for Loop 
# Store each new data point in 'adequate' folder 
# Store each file's date and time in 'datacalendar' folder 

parent=Path('data')
adequate=[]
datacalendar=[]

for item in assets:
    ds=xr.open_dataset(parent/item, group='geophysical_data', engine='netcdf4', format=None, decode_coords=None)
    cldice_flag_idx = ds['l2_flags'].attrs['flag_meanings'].split().index('CLDICE')
    cldice_flaged_mask = ds['l2_flags'].attrs['flag_masks'][cldice_flag_idx]
    ds['cldice'] = np.bitwise_and(ds['l2_flags'], cldice_flaged_mask)//cldice_flaged_mask
    tw=xr.open_dataset(parent/item)
    date=tw.attrs['time_coverage_start']
    date=datetime.fromisoformat(date[:-1])
    datacalendar.append(date)
    bad=ds['cldice'].sum()
    total=ds['cldice'].count()
    adequate.append(bad/total)


In [ ]:
# To visualize our new data points we can use hvplot 
ds=xr.concat(adequate,dim='date')
ds=ds.assign_coords({'date':datacalendar})
ds.hvplot.line(xlabel='Date',ylabel='Proportion of Cloud-Covered Pixels', title='Sable Island L2 Aqua-MODIS Cloud Data')
        

In [ ]:
# Observations for date ranging April 12th to May 2nd 2016, location: Sable Island 

# The proportion of cloud-covered pixels follows a daily trend of increasing as the morning begins and peaking around the late afternoon 
# The proportion of cloud-covered pixels follows a daily trend of decreasing after this late afternoon peak until it reaches a minimum in the early morning
# The lowest proportion of cloud-covered pixels ranges daily from around 3 am to 10 am 
# The highest proportion of cloud-covered pixels ranges daily from around 3 pm to 8 pm 

# Analysis 

# The most clear satellite images(lowest proportion of cloud-covered pixels) occur daily in the early morning
# The most unclear satellite images(highest proportion of cloud-covered pixels) occur daily in the evening

In [ ]:
# Below depicts a visual for the chlorophyll concentration and cloud-coverage around Sable Island for April 20th
path = 'data/A2016111170000.L2_LAC_OC.nc'
ds = xr.open_dataset(path, group='navigation_data')
geo = ds[['longitude', 'latitude']]
geo = geo.rename_dims({'pixel_control_points': 'pixels_per_line'})
ds = xr.open_dataset(path, group='geophysical_data')
ds = xr.Dataset(ds[['chlor_a', 'l2_flags']], coords=geo)
cldice_flag_idx = ds['l2_flags'].attrs['flag_meanings'].split().index('CLDICE')
cldice_flaged_mask = ds['l2_flags'].attrs['flag_masks'][cldice_flag_idx]
ds['cldice'] = np.bitwise_and(ds['l2_flags'], cldice_flaged_mask)//cldice_flaged_mask

In [ ]:
# This is the chlorophyll concentration image
hv.extension('bokeh')

ds.hvplot.quadmesh(x='longitude', y='latitude', z='chlor_a', rasterize=True)

In [ ]:
# This is the cloud covered image 
hv.extension('bokeh')

ds.hvplot.quadmesh(x='longitude', y='latitude', z='cldice', rasterize=True)